# 3547 TERM PROJECT

## Intelligent Systems and Reinforcement Learning

### Title: Robot obstacle avoidance with reinforcement learning

#### Group Members:
    1. Alexandre Dietrich
    2. Ankur Tyagi
    3. Haitham Alamri
    4. Rodolfo Vasconcelos


Date: August, 4 2020

# Table of Contents

* [Introduction](#Introduction)
* [Problem](#Problem)
* [Solution](#Solution)
    * [Alternative 1](#Alternative_1)
        * [Training Phase](#Training_Phase)
        * [Playing Phase](#Playing_Phase)
    * [Alternative 2](#Alternative_2)
        * [Description](#Description)
        * [Evaluation](#Evaluation)
* [Conclusion](#Conclusion)
* [Appendix A](#Appendix_A)

<a id='Introduction'></a>
# Introduction

After ten weeks of learning additional Artificial Intelligence techniques, which cover more traditional branches of AI, such as research, planning, knowledge, logic and reinforcement learning, we had the opportunity to put what we learned into practice. After interesting brainstorming and group discussions, we decided to go ahead with a project using reinforcement learning. We found a challenging project and the right size to carry out a solution in a reasonable time and resources. We then put our efforts into defining the problem, defining workarounds, coding and testing our algorithms, evaluating the results and preparing for the conclusion. Here's what we did.

<a id='Problem'></a>
# Problem

### Problem statement: find the optimal path for a robot to reach an end point, starting from anothe point, avoiding all obstacles during its journey and using reinforcement learning methods.

We found an implementation of this challenge that uses other techniques, instead of reinforcement learning. Here is the link to the article that inspired us on this project. 

https://bayesianadventures.wordpress.com/2015/08/31/obstacle-avoidance-for-clever-robots/

For this project, the environment created by the original simulator is called "Robot World".

<div align="center">
<img src="original_simulator.png" alt="Drawing" style="width: 800px;"/>
<center><b>Figure 1. Original Robot World Simulator</b></center>
<center><i>Source: Bayesian Adventures</i></center>
</div> 

In this Robot World, you can move around by clicking on the step button. After a new step, the robot will be in a new state. You can add new obstacles during the journey dynamically. The objective is to reach the goal point (G) without colliding with an obstacle. You can also define the starting and goal points. 

The original code implemented three important classes: Robot, Sonar and Sonar_Array. The first one controls the robot, its position, its movements, etc. The other two implement the controls to obtain information from all the sensors and allows the robot to uderstantd if there are obstacles in close range, define the alerts and define the actions that control whether the robot should continue on its path or need to make a turn to avoid an obstacle. This part of the code is where our main problem lies. 

- Can we use reinforcement learning to define an optimal policy to guide the robot's actions to avoid the obstacles and reach the goal? 
- Can we replace the original algorithm by a more efficient RL algorithm or RL is not the best solution for this problem? 
- Can we extrapolate this problem to a drone or to a more dynamic environment ? 

These were some of the questions that guided our project. 

The original code and simultador can be see and run in this link: <br>
http://www.codeskulptor.org/#user40_EEIxkOtKog_1.py

     
To effectively use the original code and simulator, we made some changes and adapted the Robot World. Our goal was to focus more on reinforcement learning algorithms than on other parts of the code. 
<div align="center">
<img src="new_simulator.png" alt="Drawing" style="width: 800px;"/>
<center><b>Figure 2. New Robot World Simulator</b></center>
<center><i>Source: New Robot World from Term Project</i></center>
</div> 

# Solution

We tried different alternatives to solve the problem. To effectively use the original code and simulator, we made some changes and adapted the Robot World. Our main goal was to focus more on reinforcement learning algorithms than on other parts of the code, but we also adapted the original code to better compare the results of our alternatives. Below you can see our solution alternatives. 

<a id='Alternative_1'></a>
## Alternative 1

In our first approach, we redefined the Robot World like this:

New Robot World Assumptions: <br>
500 x 500 pixels <br>
Start point at the top left corner <br>
End point at the bottom right corner <br>

<div align="center">
<img src="new_simulator.png" alt="Drawing" style="width: 800px;"/>
<center><b>Figure 2. New Robot World Simulator</b></center>
<center><i>Source: New Robot World from Term Project</i></center>
</div> 

Following is the definition of the Markov decision process: 

States: all possible states within a 4 x 4 grid with 0, 1 or 2 obstacles<br>
Start State: top left corner<br>
Actions(s): Up, Down, Right and left<br>
T(s'|s; a): probability of s0 if take action a in state s = 1 (No uncertainty)<br>
Reward(s; a; s0): -1 without obsctacle, -5 when there is an obstacle<br>
End State: bottom right corner <br>
Discount factor = 0.6<br>

<a id='Training_Phase'></a>
### Training Phase


To find an optimal policy to help our robot reach its goal without hitting obstacles, we made an operational decision and divided the New Robot World (500 x 500) in 100 4 x 4 position grids. With that, we created a strategy to find the best policy in a smaller grid, considering that we could have 15 different positions for one or two obstacles. We ran the Monte Carlo method to simulate thousands of episodes and value iteration to find the optimal q-value function for every state and action within the smaller grid to define the optimal policy (master policy). This same policy could be used for all other 99 grids. 

After that initial phase to define an optimal policy within this 4 x 4 grid, we just copied the print of policy to the code that implement the playing stage which was adapted to run the New Robot World. 

# Insert below the code that generates the Master Policy  - We need to decide how much of the code we want to show here ****

In [2]:
# Insert here any code to show Monte Carlo

<a id='Playing_Phase'></a>
### Playing Phase

With the Master Policy trained, we adapted the original code of the Robot World to create our New Robot World. We included the Master Policy as a dictionary considering States and Actions. We also created two functions: find_location_onMap and policy_finder, which are use to obtain information of the robot and obstacles within a specific grid and define the optimal action according the these positions. 

Within the code where the original program understands what action the robot should take, we changed the code to call our policy_finder fuction, which find the robot and obstacle locations, and return the action that the robot must take, according to our Master Policy. 

You can now play with the New Robot World by clicking on the step button and adding new obstacles. 

Here is the link to the New Robot World:

http://www.codeskulptor.org/#user47_JJSgNIMtcq_0.py

Following are parts of the code that were inserted or updated int the original simulator for the New Robot World.

# Insert below the code that plays with the robot to avoid obstacles with RL

In [ ]:

# New Code
# Add the master pilocy 
master_policy={0: {0: {(0, 0): 'R', (0, 1): 'R', (0, 2): 'R', (0, 3): 'D', (1, 0): 'D', (1, 1): 'L', (1, 2): 'U', (1, 3): 'D', (2, 0): 'R', (2, 1): 'D', (2, 2): 'D', (2, 3): 'D', (3, 0): 'R', (3, 1): 'R', (3, 2): 'R', (3, 3): 'U'}, 1: {(0, 0): 'R', (0, 1): 'R', (0, 2): 'R', (0, 3): 'D', (1, 0): 'D', (1, 1): 'R', (1, 2): 'R', (1, 3): 'D', (2, 0): 'D', (2, 1): 'L', (2, 2): 'R', (2, 3): 'D', (3, 0): 'R', (3, 1): 'R', (3, 2): 'R', (3, 3): 'U'}, 2: {(0, 0): 'R', (0, 1): 'R', (0, 2): 'D', (0, 3): 'D', (1, 0): 'U', (1, 1): 'R', (1, 2): 'R', (1, 3): 'D', (2, 0): 'R', (2, 1): 'D', (2, 2): 'R', (2, 3): 'D', (3, 0): 'U', (3, 1): 'R', (3, 2): 'R', (3, 3): 'U'}, 3: {(0, 0): 'R', (0, 1): 'R', (0, 2): 'R', (0, 3): 'D', (1, 0): 'U', (1, 1): 'R', (1, 2): 'R', (1, 3): 'D', (2, 0): 'U', (2, 1): 'D', (2, 2): 'R', (2, 3): 'D', (3, 0): 'R', (3, 1): 'R', (3, 2): 'R', (3, 3): 'U'}}, 1: {0: {(0, 0): 'R', (0, 1): 'R', (0, 2): 'R', (0, 3): 'D', (1, 0): 'R', (1, 1): 'D', (1, 2): 'R', (1, 3): 'D', (2, 0): 'D', (2, 1): 'D', (2, 2): 'D', (2, 3): 'D', (3, 0): 'R', (3, 1): 'R', (3, 2): 'R', (3, 3): 'U'}, 1: {(0, 0): 'D', (0, 1): 'L', (0, 2): 'R', (0, 3): 'D', (1, 0): 'D', (1, 1): 'D', (1, 2): 'D', (1, 3): 'D', (2, 0): 'D', (2, 1): 'D', (2, 2): 'D', (2, 3): 'D', (3, 0): 'R', (3, 1): 'R', (3, 2): 'R', (3, 3): 'U'}, 2: {(0, 0): 'D', (0, 1): 'L', (0, 2): 'L', (0, 3): 'D', (1, 0): 'D', (1, 1): 'D', (1, 2): 'R', (1, 3): 'D', (2, 0): 'R', (2, 1): 'D', (2, 2): 'L', (2, 3): 'D', (3, 0): 'R', (3, 1): 'R', (3, 2): 'R', (3, 3): 'U'}, 3: {(0, 0): 'R', (0, 1): 'D', (0, 2): 'D', (0, 3): 'L', (1, 0): 'D', (1, 1): 'D', (1, 2): 'D', (1, 3): 'L', (2, 0): 'D', (2, 1): 'R', (2, 2): 'R', (2, 3): 'D', (3, 0): 'R', (3, 1): 'R', (3, 2): 'R', (3, 3): 'U'}}, 2: {0: {(0, 0): 'R', (0, 1): 'R', (0, 2): 'D', (0, 3): 'D', (1, 0): 'R', (1, 1): 'R', (1, 2): 'D', (1, 3): 'D', (2, 0): 'D', (2, 1): 'R', (2, 2): 'R', (2, 3): 'D', (3, 0): 'R', (3, 1): 'R', (3, 2): 'R', (3, 3): 'U'}, 1: {(0, 0): 'R', (0, 1): 'R', (0, 2): 'R', (0, 3): 'D', (1, 0): 'U', (1, 1): 'R', (1, 2): 'D', (1, 3): 'D', (2, 0): 'D', (2, 1): 'L', (2, 2): 'R', (2, 3): 'D', (3, 0): 'R', (3, 1): 'R', (3, 2): 'R', (3, 3): 'U'}, 2: {(0, 0): 'R', (0, 1): 'R', (0, 2): 'R', (0, 3): 'D', (1, 0): 'D', (1, 1): 'U', (1, 2): 'R', (1, 3): 'D', (2, 0): 'D', (2, 1): 'D', (2, 2): 'R', (2, 3): 'D', (3, 0): 'R', (3, 1): 'R', (3, 2): 'R', (3, 3): 'U'}, 3: {(0, 0): 'R', (0, 1): 'D', (0, 2): 'D', (0, 3): 'L', (1, 0): 'U', (1, 1): 'D', (1, 2): 'D', (1, 3): 'L', (2, 0): 'U', (2, 1): 'R', (2, 2): 'D', (2, 3): 'D', (3, 0): 'R', (3, 1): 'R', (3, 2): 'R', (3, 3): 'U'}}, 3: {0: {(0, 0): 'R', (0, 1): 'R', (0, 2): 'D', (0, 3): 'D', (1, 0): 'U', (1, 1): 'U', (1, 2): 'D', (1, 3): 'D', (2, 0): 'U', (2, 1): 'D', (2, 2): 'R', (2, 3): 'D', (3, 0): 'R', (3, 1): 'R', (3, 2): 'R', (3, 3): 'U'}, 1: {(0, 0): 'R', (0, 1): 'R', (0, 2): 'R', (0, 3): 'D', (1, 0): 'U', (1, 1): 'D', (1, 2): 'R', (1, 3): 'D', (2, 0): 'R', (2, 1): 'R', (2, 2): 'D', (2, 3): 'D', (3, 0): 'U', (3, 1): 'R', (3, 2): 'R', (3, 3): 'U'}, 2: {(0, 0): 'R', (0, 1): 'R', (0, 2): 'D', (0, 3): 'D', (1, 0): 'R', (1, 1): 'R', (1, 2): 'R', (1, 3): 'D', (2, 0): 'U', (2, 1): 'U', (2, 2): 'R', (2, 3): 'D', (3, 0): 'R', (3, 1): 'U', (3, 2): 'R', (3, 3): 'U'}, 3: {(0, 0): 'D', (0, 1): 'D', (0, 2): 'R', (0, 3): 'D', (1, 0): 'R', (1, 1): 'R', (1, 2): 'R', (1, 3): 'D', (2, 0): 'U', (2, 1): 'D', (2, 2): 'D', (2, 3): 'D', (3, 0): 'U', (3, 1): 'R', (3, 2): 'R', (3, 3): 'U'}}}
master_policy_2obs={0: {0: {0: {0: {(0, 0): 'R', (0, 1): 'D', (0, 2): 'L', (0, 3): 'D', (1, 0): 'R', (1, 1): 'D', (1, 2): 'D', (1, 3): 'D', (2, 0): 'R', (2, 1): 'D', (2, 2): 'R', (2, 3): 'D',

# New Code

                                    
# this function to find the location of agent or obtacles in the map 
# it convert 500X500 pixels word to 10x10 squars each with 12.5x12.5 pixxels
def find_location_onMap(pos):
  location_in_the_grid=[]
  location_in_the_map=[]
  i=0
  i1=0
  loc=0
  loc1=0
  for squares in range(0,10):
    i1=0
    for quare in range(0,10):
      loc=0
      #square.append((i,i1))
      if ((pos[0]>i and pos[0]<=i+50) and (pos[1]>i1 and pos[1]<=i1+50)):
        for rows in range(0,4):
          loc1=0
          for col in range(0,4):
            if ((pos[0]>loc+i and pos[0]<=loc+i+12.5) and (pos[1]>loc1+i1 and pos[1]<=loc1+12.5+i1)):           
              location_in_the_grid.append(loc)
              location_in_the_grid.append(loc1)
              location_in_the_map.append(i)
              location_in_the_map.append(i1)
            
            loc1+=12.5
          loc+=12.5
      i1+=50
    i+=50

#To to convert to 4x4 grid each is 12.5 X 12.5 pixels
  location_in_the_grid[0]=int(location_in_the_grid[0]/12.5)
  location_in_the_grid[1]=int(location_in_the_grid[1]/12.5)
#To to convert to 5x5 map each sqaure is 100X100 pixel
  location_in_the_map[0]=int(location_in_the_map[0]/50)
  location_in_the_map[1]=int(location_in_the_map[1]/50)
  return location_in_the_map, location_in_the_grid

# this function to get the policy and find the action fro the agen as per the obstacle location in 4x4 gris
def policy_finder (mylocation,obs):
  if(obs):
    print(obs)
    mylocation_onMap, my_location_onGrid = find_location_onMap(mylocation)
    obs_location_onMap, obs_location_onGrid = find_location_onMap(obs[0])

    policy= master_policy[obs_location_onGrid[0]][obs_location_onGrid[1]]
    direction = policy.get((my_location_onGrid[0],my_location_onGrid[1]), ' ')
    print(direction)
    return direction

def policy_finder2_obs (mylocation,obs):
  mylocation_onMap, my_location_onGrid = find_location_onMap(mylocation)
  obs_location_onMap, obs_location_onGrid = find_location_onMap(obs[0])
  obs_location_onMap1, obs_location_onGrid1 = find_location_onMap(obs[1])
  policy= master_policy_2obs[obs_location_onGrid[0]][obs_location_onGrid[1]][obs_location_onGrid1[0]][obs_location_onGrid1[1]]
  direction = policy.get((my_location_onGrid[0],my_location_onGrid[1]), ' ')
  print("kokokokoo",direction)
  return direction
  
# This function is to check if the obtacles and agent are in the same 12.5x12.5 pixels sqaure
def check_obstacle(pos, obs_list):
  obstacles=[]
  location_on_map,location_on_grid = find_location_onMap(pos)
  for i in obs_list:
    location_on_map1,location_on_grid1 = find_location_onMap(i)
    if (location_on_map1 == location_on_map):
        obstacles.append(i)
  return obstacles
  return None
                                    

    def weighted_sum_method(self, robot_pos, robot_co):
        #process data by the weighted sum method and 
        #return (1) whether turn is required or not (2) index of recommended sonar LOS to turn to
        sum_d = 0
        sum_wt = 0
        alert = False
        obs=[]
        
        obs=check_obstacle(robot_pos,full_obstacle_list)
        print("---------------------------")
        print("I see obstacles")
        print("---------------------------")
        #print "checking all sonars:"      
        for sonar in self.sonar_list:
            #print "sonar:", sonar.index, " range:", sonar.output
            if sonar.output < SENSOR_ALERT_R:#has this sonar found anything in danger zone?
                alert = True
                #print "obstacle found by index ", sonar.index
                for s1 in self.sonar_list: #process the whole array
                    d = int(s1.output)
                    gain = 1#SENSOR_MAX_R/(SENSOR_MAX_R - d)
                    sum_d +=  d
                    sum_wt += s1.index * d * gain
                    #print "I:", s1.index,",D:",int(s1.output), ",sum_D:", sum_d, "sum_wt:",sum_wt
                rec_index = math.ceil(TURN_SCALE_FACTOR * float(sum_wt)/sum_d) #index of sonar with best LOS
                #rec_index = int(TURN_SCALE_FACTOR * float(sum_d)/sum_wt)
                print "Rec index:", rec_index
                if abs(rec_index) > n_sensor/2:
                    print "rec index too large"
                    rec_index = n_sensor/2
#                    if rec_index < 0:
#                        rec_index = -n_sensor/2
#                    else:
#                        rec_index = n_sensor/2
                print "Rec index:", rec_index 
                #break # processing completed
            else: #no obstacle in danger zone
                rec_index = 0
                #return robot_co, False
                #print : index = ", sonar.index
        #print "break from loop."
        if rec_index == 0 and alert == True:
                print "alert with no alteration"
                obs=check_obstacle(robot_pos,full_obstacle_list)
                print(obs)
               
                if (len(obs)!=0 and len(obs)<2):
                        action = policy_finder(robot_pos,obs)
                        I_was_here.append(robot_pos)
                elif(len(obs)==2):
                        action = policy_finder2_obs(robot_pos,obs)
                        I_was_here.append(robot_pos)
                else:
                        return robot_co, False
                if action:
                        print(action,"HAAH")
                        if action == 'R':
                            print "policy recommend to go right "
                            offset=90
                        elif action == 'D':
                            offset=180
                            print "policy recommend to go down "
                        elif action == 'L':
                            offset=270
                            print "policy recommend to go left "
                        elif action == 'U':
                            offset = 359
                            print "policy recommend to go down "
                        #if (I_was_here[-2]==robot_pos):
                            #return robot_co+45, True
                        print("Robot positon",robot_pos)
                        print("*********")
                        print("Robot Co",robot_co)
                        print("New Direction",(offset%robot_co)+robot_co)
                        print("******")                        
                        return offset, True
                        #return (offset%robot_co)+robot_co, True 
        
                        
                
        elif abs(rec_index) > 0: # some alteration recommended
           obs=[]
           offset =  rec_index * SENSOR_FOV #how much is the angular offset  
           print(obs)
           obs=check_obstacle(robot_pos,full_obstacle_list)
           if(obs!=None):
            
                if (len(obs)!=0  and len(obs)<2):
                            action = policy_finder(robot_pos,obs)
                            I_was_here.append(robot_pos)
                           
                elif(len(obs)==2):
                            action = policy_finder2_obs(robot_pos,obs)
                            I_was_here.append(robot_pos)
                            
                else:
                            return robot_co, False
                if action:
                            if action == 'R':
                                print "policy recommend to go right "
                                offset=90
                            elif action == 'D':
                                offset=180
                                print "policy recommend to go down "
                            elif action == 'L':
                                offset=270
                                print "policy recommend to go left "
                            elif action == 'U':
                                offset = 359
                                print "policy recommend to go down "
                            #if (I_was_here[-2]==robot_pos):
                                    #return robot_co+45, True
                print("Robot positon",robot_pos)
                #print("obs positon",obs)
                print("*********")
                print("Robot Co",robot_co)
                print("New Direction",(offset%robot_co)+robot_co)
                #print("location in the grid:",find_location_onMap(robot_pos))
                #print("location in the grid:",find_location_onMap(obs[0]))
                print("******")
                return offset,True
                #return (offset%robot_co)+robot_co, True
           else:     
            
                return (offset%robot_co), True
        else:# no diversion needed
           obs=[]
           offset =  rec_index * SENSOR_FOV #how much is the angular offset  
           print(obs)
           obs=check_obstacle(robot_pos,full_obstacle_list)
           if(obs!=None):
            
                if (len(obs)!=0  and len(obs)<2):
                            action = policy_finder(robot_pos,obs)
                            I_was_here.append(robot_pos)
                            print("chinca:1")
                elif(len(obs)==2):
                            action = policy_finder2_obs(robot_pos,obs)
                            I_was_here.append(robot_pos)
                            print("chinca:2")
                else:
                            return robot_co, False
                if action:
                            if action == 'R':
                                print "policy recommend to go right "
                                offset=90
                            elif action == 'D':
                                offset=180
                                print "policy recommend to go down "
                            elif action == 'L':
                                offset=270
                                print "policy recommend to go left "
                            elif action == 'U':
                                offset = 359
                                print "policy recommend to go down "
                            #if (I_was_here[-2]==robot_pos):
                                    #return robot_co+45, True
                print("Robot positon",robot_pos)
                #print("obs positon",obs)
                print("*********")
                print("Robot Co",robot_co)
                print("New Direction",(offset%robot_co)+robot_co)
                #print("location in the grid:",find_location_onMap(robot_pos))
                #print("location in the grid:",find_location_onMap(obs[0]))
                print("******")
                return offset,True
                #return (offset%robot_co)+robot_co, True


The results of this alternative were not good enough. We had too many assumptions like fix start and end points, no more than two obstacles per 4 x 4 grids, two different phases (training and playing) and the could only play the game online clicking on the step button.  

<a id='Alternative_2'></a>
## Alternative 2

For our second alternative, we decided to keep the same strategy as alternative 1 of dividing the original world of 500 x 500 into 100 4 x 4 grids. In this alternative, we allowed dynamic start and end points and no fix number of obstacles within a 4 x 4 grid. 

<a id='Description'></a>
### Description

Following is the definition of the Markov decision process: 

States: all possible states within a 4 x 4 grid with 0 to 15 obstacles<br>
Start State: dynamic, defined when the episode starts<br>
Actions(s): Up, Down, Right and left<br>
T(s'|s; a): probability of s0 if take action a in state s = 1 (No uncertainty)<br>
Reward(s; a; s0): -1 without obsctacle, -5 when there is an obstacle<br>
End State: dynamic, defined when the episode starts <br>
Discount factor = 0.6<br>

To compare the performance to the original code and our solution, we converted the original code into Python 3, broked it in 4 Python programs to implement the three classes (Robot, Sonar and Sonar_Array) and utilitarian functions. This was important to run multiples episodes of the Robot in a bacth mode. 

Our reinforcement learning strategy is now considering a dynamic policy. We implemented just one phase (playing), starting every episode without a policy. For every step (action) taken by our robot in the New Robot World, if a policy is not available for that state, a new policy is created using the same Monte Carlo method used in the alternative 1. Then the policy for that state is  stored. If the robot enters a state where a possible had been previously defined, then the robot uses that policy. The array with the policies is being appended as long as the robot moves step by step until it reaches the goal. 

Our new robot, supported now by reinforcement leanirng instead of bayesian methods, can run in the same graphical interface than the original robot. However, to compare the performance of the two robots, we create a code to run both robots simultaneous, with the same number and position of obstacles, and to capture the number of steps each robot takes to reach the goal after 200 episodes. Then, we can plot two graphs to compare the accuracy and effort of both robots. 

## Insert below the code for alternatives: different RL methods, new assuptions, etc. 

In [ ]:
### Monte Carlo Method

"""## Run all episodes"""
def calculate_gridworld_policy(end_state=(3,3),obstable_list = []):
  # use the standard grid again (0 for every step) so that we can compare
  # to iterative policy evaluation
  # grid = standard_grid()
  # try the negative grid too, to see if agent will learn to go past the "bad spot"
  # in order to minimize number of steps
  grid = negative_grid(step_cost=-1)
  update_rewards(grid, obstable_list, -5)
  update_end_state(grid, end_state)

  # print rewards
  #print("rewards:")
  print_values(grid.rewards, grid)

  pi = defaultdict(lambda: 1/len(ALL_POSSIBLE_ACTIONS))  # probability of action (def random)

  # state -> action
  # initialize a random policy
  policy = {}
  for s in grid.actions.keys():
      policy[s] = np.random.choice(ALL_POSSIBLE_ACTIONS)

  # initialize Q(s,a) and returns
  Q = {}
  returns = {} # dictionary of state -> list of returns we've received
  states = grid.all_states()
  for s in states:
      if s in grid.actions: # not a terminal state
        Q[s] = {}
        for a in ALL_POSSIBLE_ACTIONS:
          Q[s][a] = -10
          returns[(s,a)] = []
  else:
      # terminal state or state we can't otherwise get to
      pass

  #print("initial Q:")
  #print_Q(Q,grid)

  # repeat until convergence
  deltas = []
  for t in range(EPISODES):
      #if t % 1000 == 0:
          #print(t)
          #print("Q:")
          #print_Q(Q,grid)

      # generate an episode using pi
      biggest_change = 0
      states_actions_returns = play_episode(grid, policy, pi)

      # calculate Q(s,a)
      seen_state_action_pairs = set()
      for s, a, G in states_actions_returns:
          # check if we have already seen s
          # called "first-visit" MC policy evaluation
          sa = (s, a)
          if sa not in seen_state_action_pairs:
              old_q = Q[s][a]
              returns[sa].append(G)
              Q[s][a] = np.mean(returns[sa])
              biggest_change = max(biggest_change, np.abs(old_q - Q[s][a]))
              seen_state_action_pairs.add(sa)
              A_star, _ = max_dict(Q[s])
              for a_index in ALL_POSSIBLE_ACTIONS:
                  if a_index == A_star:   pi[(s,a_index)] = 1 - EPS + EPS/len(ALL_POSSIBLE_ACTIONS)
                  else:                   pi[(s,a_index)] = EPS/len(ALL_POSSIBLE_ACTIONS)

      deltas.append(biggest_change)

      # calculate new policy pi(s) = argmax[a]{ Q(s,a) }
      for s in policy.keys():
          a, _ = max_dict(Q[s])
          policy[s] = a

  """## Print results"""

  #plt.plot(deltas)
  #plt.show()

  # find the optimal state-value function
  # V(s) = max[a]{ Q(s,a) }
  V = {}
  for s in policy.keys():
      V[s] = max_dict(Q[s])[1]

  #print("final values:")
  #print_values(V, grid)
  #print("final policy:")
  #print_policy(policy, grid)
  #print("final Q:")
  #print_Q(Q,grid)
  return policy


In [ ]:
### Calculate a policy or use one that is already calculated

def dynamic_policy_finder (mylocation, obs, master_policy, goal_pos):
    print(f"mylocation={mylocation}, obs={obs}, goal_pos={goal_pos}")
    
    mylocation_onMap, my_location_onGrid = find_location_onMap(mylocation)
    print(f"mylocation_onMap={mylocation_onMap}, my_location_onGrid={my_location_onGrid}")

    obs_location_onGrid_array = []

    for obstacle_pos in obs:
        #_, obs_location_onGrid = find_location_onMap(obstacle_pos)
        #obs_location_onGrid_array.append((obs_location_onGrid[0],obs_location_onGrid[1]))
        obs_location_onGrid_array.extend(calculate_obstacle_onGrid(obstacle_pos))
        print(f"obs_location_onGrid_array={obs_location_onGrid_array}")
    
    end_state = calculate_end_state_onGrid(mylocation, obs_location_onGrid_array, goal_pos)

    policy_key = f"{end_state}|{obs_location_onGrid_array}"
    print(f"policy_key={policy_key}")

    if policy_key in master_policy:
        policy = master_policy[policy_key]
    else:
        policy = montecarlo.calculate_gridworld_policy(end_state, obs_location_onGrid_array)
        master_policy[policy_key] = policy

    #policy= master_policy[obs_location_onGrid[0]][obs_location_onGrid[1]]
    direction = policy.get((my_location_onGrid[0],my_location_onGrid[1]), ' ')
    print(f"direction={direction}")
    return direction



In [ ]:
### Class Sonar_Array

    def weighted_sum_method(self, robot_pos, robot_co,full_obstacle_list,master_policy,I_was_here,goal_pos)

        print (f"weighted_sum_method, robot_pos={robot_pos}, robot_co={robot_co}")

        sum_d = 0
        sum_wt = 0
        alert = False
        obs=[]
        
        obs=check_obstacle(robot_pos,full_obstacle_list)
               
        action = utils.dynamic_policy_finder(robot_pos,obs,master_policy,goal_pos)
        print(f"action={action},")
        if action == 'R':
            print ("policy recommend to go right ")
            offset=90
        elif action == 'D':
            offset=180
            print ("policy recommend to go down ")
        elif action == 'L':
            offset=270
            print ("policy recommend to go left ")
        elif action == 'U':
            offset = 359
            print ("policy recommend to go down ")
        else:
            offset = 0
            print ("no policy?")
        #if (I_was_here[-2]==robot_pos):
            #return robot_co+45, True
        print("Robot positon",robot_pos)
        print("*********")
        print("Robot Co",robot_co)
        print("New Direction",(offset%robot_co)+robot_co)
        print("******")                        
        return offset, True


### Evaluation

Describe the aspects of the performance of the algoritms

<div align="center">
<img src="WhatsApp Image 2020-08-01 at 13.30.50.jpeg" alt="Drawing" style="width: 800px;"/>
<center><b>Figure 4. Performance of both Robots</b></center>
<center><i>Source: Term Project</i></center>
</div> 

# Conclusion

Describe the conclusions

<div align="center">
<img src="robot_movements.png" alt="Drawing" style="width: 800px;"/>
<center><b>Figure 3. Final Robot Movements with RL</b></center>
<center><i>Source: XXXXXX</i></center>
</div> 

<a id='Appendix_A'></a>
# Appendix A

All the code prepared for this project is available on Github. Here is the link for the project. 

https://github.com/ravasconcelos/rl_obstacle_avoidance